# PYTHON. Calculations for AB test analysis

In [1]:
import scipy.stats as st
import math
import numpy as np
import pandas as pd

## Basic theoretical confidence interval calculation
Assumptions: Normal distribution (functions from **scipy.stats** library)



In [2]:
def conf_interval(clicks, N, conf=0.95, n_tails=2, rnd=4):
    # ppf(q, loc=0, scale=1) Percent point function (inverse of cdf — percentiles).
    z=st.norm.ppf(1-(1-conf)/n_tails)

    p_hat=clicks/N
    # binomial distribution
    standard_error= math.sqrt(p_hat*(1-p_hat)/N)

    margin_of_error = standard_error * z

    interval_min = p_hat - margin_of_error
    interval_max = p_hat + margin_of_error

    #st.norm.cdf(z)
    print(f'P_hat:           {round(p_hat,4)}')
    print(f'Standard error:  {round(standard_error,4)}')
    print(f'Margin of error: {round(margin_of_error,4)}')
    print("-"*20)
    print(f"{round(conf*100,1)}% {n_tails}-tail confidence interval is [{round(interval_min,rnd)} ; {round(interval_max,rnd)}]")



In [3]:
N=2000
clicks=300
conf=0.99
n_tails=2

conf_interval(clicks, N, conf=conf, n_tails=n_tails)

P_hat:           0.15
Standard error:  0.008
Margin of error: 0.0206
--------------------
99.0% 2-tail confidence interval is [0.1294 ; 0.1706]


## Sample size
How many observations (at least) need to be collected to have significant results?

* [Even Miller calculator](https://www.evanmiller.org/ab-testing/sample-size.html)
* calculation based on pooled standard error

In [4]:
#https://towardsdatascience.com/mathematical-intuition-behind-a-b-testing-with-python-9d024e5e7f37

def calc_sample_size(alpha, beta, p, delta, method, n_tails=2):
    """ Based on https://www.evanmiller.org/ab-testing/sample-size.html
    Ref: https://stats.stackexchange.com/questions/357336/create-an-a-b-sample-size-calculator-using-evan-millers-post
    Args:
        alpha (float): How often are you willing to accept a Type I error (false positive)?
        power (float): How often do you want to correctly detect a true positive (=1-beta)?
        p (float): Base conversion rate
        pct_mde (float): Minimum detectable effect, relative to base conversion rate.
    """
    if method == 'evanmiller':
        t_alpha2 = st.norm.ppf(1.0-alpha/n_tails)
        t_beta = st.norm.ppf(1-beta)

        sd1 = np.sqrt(2 * p * (1.0 - p))
        sd2 = np.sqrt(p * (1.0 - p) + (p + delta) * (1.0 - p - delta))

        n= round((t_alpha2 * sd1 + t_beta * sd2) * (t_alpha2 * sd1 + t_beta * sd2) / (delta**2))
    elif method == 'pooled_se':
        """
        References:
            Code taken from Nguyen Ngo: https://towardsdatascience.com/the-math-behind-a-b-testing-with-example-code-part-1-of-2-7be752e1d06f
            Stanford lecture on sample sizes     
        """
        # standard normal distribution to determine z-values
        standard_norm = st.norm(0, 1)

        # find Z_beta from desired power
        Z_beta = standard_norm.ppf(1-beta)

        # find Z_alpha
        Z_alpha = standard_norm.ppf(1-alpha/n_tails)

        # average of probabilities from both groups
        pooled_prob = (p + p+delta) / 2

        n= round(2*(pooled_prob * (1 - pooled_prob) * (Z_beta + Z_alpha)**2
                 / delta**2))
    return n



In [5]:
alpha=0.05
beta=0.2
power=1-beta
d_min=0.02 # delta
p_hat=0.1
pct_mde=p_hat-d_min/p_hat
delta=d_min

n_evanmiller=calc_sample_size(alpha, beta, p_hat, d_min, method='evanmiller')
n_stanford=calc_sample_size(alpha, beta, p_hat, d_min, method='pooled_se')

print (f"Sample size \n1) Evan Miller: {n_evanmiller}  \n2) Pooled_se:  {n_stanford} \nAverage: {round(np.mean([n_stanford,n_evanmiller]))}")


Sample size 
1) Evan Miller: 3623  
2) Pooled_se:  3842 
Average: 3732


## AB test experiment analysis

Pooled Standard error theoretical confidence interval calculation and comparison to practical minimal uplift. 
Verdict on the further actions based on the results

In [6]:
#theoretical
def ab_pooled_theor_sign(N_cont, action_cont, N_exp, action_exp, d_min, conf=0.95, n_tails=2):
    print(f"Control \n N      {N_cont}\n action {action_cont}")
    print(f"Experiment \n N      {N_exp}\n action {action_exp}")
    print("-"*20)
    
    p_hat_cont=action_cont/N_cont
    print(f"P control:    {round(p_hat_cont,4)}")

    p_hat_exp=action_exp/N_exp
    print(f"P experiment: {round(p_hat_exp,4)}")

    p_hat_pool=(action_cont+action_exp)/(N_cont+N_exp)
    print(f"P poopled:    {round(p_hat_pool,4)}")

    standard_error_pool=math.sqrt(p_hat_pool*(1-p_hat_pool)*((1/N_cont) + (1/N_exp)))

    z=st.norm.ppf(1-(1-conf)/n_tails)
    margin_of_error_pool = standard_error_pool * z

    dif_hat=p_hat_exp-p_hat_cont

    interval_min=dif_hat-margin_of_error_pool
    interval_max=dif_hat+margin_of_error_pool
    print("-"*20)
    print(f'Minimal practical significance {round(d_min,4)}')
    print(f"Confidence interval [{round(interval_min,4)},{round(interval_max,4)}]")
    
    print("-"*20)
    #stat significance - interval doesn't contain zero
    if interval_min<0 and interval_max>0:
        print("Confidence interval contains zero. We cannot reject null hypotesis that there is no difference between groups")
    elif interval_min>0:
        print("There is a significant POSITIVE result")
    else:   
        print("There is a significant NEGATIVE result")


    #practical significance   - the interval is to the right from min uplift
    if d_min<=interval_min:
        print(f"With {conf*100}% confidence there is practical and significant effect greater than minimal prectical uplift ({d_min}). \nVerdict: LAUNCH the change")
    elif interval_min>=-d_min and interval_max<=d_min:
        print("There is no practical significance. \nVerdict: NO LAUNCH")
    elif interval_min<=-d_min and interval_max>=d_min:
        print ("Overlap. Results do not provide clarity. \nVerdict: Additional test may bring more solid results")
    elif interval_min>=-d_min and interval_max>=d_min:
        print ("Intersection on positive side. Results do not provide clarity. \nVerdict: Additional test may bring more solid results")   
    else: 
        print("No Launch")
    print("-"*20)

In [7]:
n_tails=2
conf=0.95
d_min=0.02
#experiment --------------------------------------------------------------
#control
N_cont=10072 # pageviews
action_cont=974 # clicks, views, etc

#experiment (target)
N_exp=9886 # pageviews
action_exp=1242 #clicks, views, etc
# calculations --------------------------------------------------------------

ab_pooled_theor_sign(N_cont, action_cont, N_exp, action_exp, d_min)
        

Control 
 N      10072
 action 974
Experiment 
 N      9886
 action 1242
--------------------
P control:    0.0967
P experiment: 0.1256
P poopled:    0.111
--------------------
Minimal practical significance 0.02
Confidence interval [0.0202,0.0376]
--------------------
There is a significant POSITIVE result
With 95.0% confidence there is practical and significant effect greater than minimal prectical uplift (0.02). 
Verdict: LAUNCH the change
--------------------


## Empirical confidence interval



In [8]:
def empir_conf_interval(data, conf=0.95, n_tails=2, rnd=4):
    N=len(data)
    data_mean=np.mean(data)
    data_std=np.std(data, ddof=1)
    data_standard_error=data_std/np.sqrt(N)

    # ppf(q, loc=0, scale=1) Percent point function (inverse of cdf — percentiles).
    z=st.norm.ppf(1-(1-conf)/n_tails)
    data_margin_of_error=z*data_standard_error


    interval_min = data_mean - data_margin_of_error
    interval_max = data_mean + data_margin_of_error
    print(f"{round(conf*100,1)}% {n_tails}-tail confidence interval is [{round(interval_min,rnd)} ; {round(interval_max,rnd)}]")

In [9]:

data=[87029, 113407, 84843, 104994, 99327, 92052, 60684]
conf=0.95
n_tails=2

empir_conf_interval(data, conf=0.95, n_tails=2, rnd=0)

95.0% 2-tail confidence interval is [79158.0 ; 104367.0]


## Variability analysis (bootstrap)

**Bootstrap** - run one bigger experiment and then randomly split this large sample of users to smaller groups, calculate metric and compare results


In [10]:
def conf_iterval_diff(data, conf=0.95, n_tails=2,rnd=4):
    # theoretical confidence interval
    N=len(data)
    data_mean=np.mean(data)
    data_std=np.std(data, ddof=1)
    print(f"Mean difference is {round(data_mean,rnd)} and standard deviation is {round(data_std,rnd)}")

    #Theoretical confidence interval
    # ppf(q, loc=0, scale=1) Percent point function (inverse of cdf — percentiles).
    z=st.norm.ppf(1-(1-conf)/n_tails)
    data_margin_of_error=z*data_std


    interval_min = data_mean - data_margin_of_error
    interval_max = data_mean + data_margin_of_error
    print("-"*20)
    print("Theoretical confidence interval")
    print(f"{round(conf*100,1)}% {n_tails}-tail confidence interval is [{round(interval_min,rnd)} ; {round(interval_max,rnd)}]")

    # Empirical confidence interval (percentile)

    ran=np.percentile(data,[2.5,97.5])
    print("-"*20)
    print("Empirical confidence interval")
    print(f"{round(conf*100,1)}% {n_tails}-tail confidence interval is [{round(ran[0],rnd)} ; {round(ran[1],rnd)}]")


In [11]:
conf=0.95
n_tails=2

# test results
g1=[0.02,0.11,0.14,0.05,0.09,0.11,0.09,0.1,0.14,0.08,0.09,0.08,0.09,0.08,0.12,0.09,0.16,0.11,0.12,0.11,0.06,0.11,0.13,0.1,0.08,0.14,0.1,0.08,0.12,0.09,0.14,0.1,0.08,0.08,0.07,0.13,0.11,0.08,0.1,0.11]
g2=[0.07,0.11,0.05,0.07,0.1,0.07,0.1,0.1,0.12,0.14,0.04,0.07,0.07,0.06,0.15,0.09,0.12,0.1,0.08,0.09,0.08,0.08,0.14,0.09,0.1,0.08,0.08,0.09,0.08,0.11,0.11,0.1,0.14,0.1,0.08,0.05,0.19,0.11,0.08,0.13]

# difference between A and B per test
g_diff=[round(x[0]-x[1],2) for x in zip(g1,g2)]

# g_diff=[0]*40
# for i in range(40):
#     g_diff[i]=round(g1[i]-g2[i],2)
   
conf_iterval_diff(g_diff, conf, n_tails)

Mean difference is 0.0042 and standard deviation is 0.0364
--------------------
Theoretical confidence interval
95.0% 2-tail confidence interval is [-0.067 ; 0.0755]
--------------------
Empirical confidence interval
95.0% 2-tail confidence interval is [-0.0605 ; 0.0802]


## Sanity check example
**Given**: We know true probability - users are assigned to A and B randomly (binomial) with probability of 0.5

**Metric**: #cookies (Unit of diversion = cookie)

**Result**: We get data by day on number of users (cookies) in each group

**Experiment duration**: 2 weeks

**Question to answer**: is the data distributed as expected = we have around 50/50 randomly distributed data 

In [12]:
#DATA PREPARATION
# data
w1=["mon", 5077,4877,'tue',5495,4729,'wed',5294,5063,'thu',5446,5035,'fri',5126,5010,'sat',3382,3193,'sun',2891,3226]
w1=np.array(w1)
df_w1=pd.DataFrame(w1.reshape(7,3),columns=['day', 'N_A', 'N_B'])
w2=["mon", 5029,5092,'tue',5166,5048,'wed',4902,5063,'thu',4923,4805,'fri',4816,4741,'sat',3411,2939,'sun',3496,3075]
w2=np.array(w2)
df_w2=pd.DataFrame(w2.reshape(7,3),columns=['day', 'N_A', 'N_B'])

for c in ['N_A', 'N_B']:
    df_w1[c]=pd.to_numeric(df_w1[c])    
    df_w2[c]=pd.to_numeric(df_w2[c]) 

df_w1['p_hat']=df_w1['N_A']/(df_w1['N_A']+df_w1['N_B'])
df_w2['p_hat']=df_w2['N_A']/(df_w2['N_A']+df_w2['N_B']) 
  

display(df_w1)
display(df_w2)

# total per week
N_w1_a=df_w1['N_A'].sum()
N_w1_b=df_w1['N_B'].sum()
N_w2_a=df_w2['N_A'].sum()
N_w2_b=df_w2['N_B'].sum()

# total for 2 weeks
N_A = N_w1_a + N_w1_b
N_B = N_w2_a + N_w2_b

print(f'Total in group A {N_A}\nTotal in group B {N_B}')

,day,N_A,N_B,p_hat
0,mon,5077,4877,0.510046
1,tue,5495,4729,0.537461
2,wed,5294,5063,0.511152
3,thu,5446,5035,0.519607
4,fri,5126,5010,0.505722
5,sat,3382,3193,0.514373
6,sun,2891,3226,0.472617


,day,N_A,N_B,p_hat
0,mon,5029,5092,0.496888
1,tue,5166,5048,0.505776
2,wed,4902,5063,0.491922
3,thu,4923,4805,0.506065
4,fri,4816,4741,0.503924
5,sat,3411,2939,0.537165
6,sun,3496,3075,0.532035


Total in group A 63844
Total in group B 62506


In [13]:
def theor_split_check (p_true, N_cont, N_exp, conf=0.95, n_tails=2, rnd=4):

    # binomial
    standard_deviation=math.sqrt(p_true*(1-p_true)/(N_cont+N_exp))
    
    
    # multiply by z score to get margin of error
    z=st.norm.ppf(1-(1-conf)/n_tails)
    margin_of_error=z*standard_deviation
    
    # compute confidence interval around 0.5
    interval_min = p_true - margin_of_error
    interval_max = p_true + margin_of_error
    
    print("Theoretical confidence interval")
    print(f"{round(conf*100,1)}% {n_tails}-tail confidence interval is [{round(interval_min,rnd)} ; {round(interval_max,rnd)}]")
    
    p_hat=N_cont/(N_cont+N_exp)
    
    
    print("-"*20)
    print(f'Observed ratio for A vs B is {round(p_hat,rnd)}')
    if p_hat>interval_min and p_hat<interval_max:
        print (f"Results ARE statistically significant with {round(conf*100,0)} confidence. Data is distributed according to expected probability")
    else:
        print (f"Results DO NOT follow expected probability with {round(conf*100,0)} confidence.")

In [14]:
p_true=0.5
conf=0.95
n_tails=2
theor_split_check (p_true, N_A, N_B, conf=0.95, n_tails=2, rnd=4)

Theoretical confidence interval
95.0% 2-tail confidence interval is [0.4972 ; 0.5028]
--------------------
Observed ratio for A vs B is 0.5053
Results DO NOT follow expected probability with 95.0 confidence.


## Single metric sanity check

**Experiment**: change color and placement of a button

**Metric**: click trough rate

**Unit of diversion**: cookie

In [15]:
#DATA
w=["d1", 51,1292, 115, 1305,
    'd2',39,853,73,835,
    'd3',64,1129,91,1133,
    'd4',43,873,60,871,
    'd5',55,1197,78,1134,
    'd6',44,1023,72,1015,
    'd7',56,1003,76,977]
w=np.array(w)
df_w=pd.DataFrame(w.reshape(7,5),columns=['day', 'click_A', 'N_A', 'click_B', 'N_B'])

totals={}
for c in [ 'click_A', 'N_A', 'click_B', 'N_B']:
    df_w[c]=pd.to_numeric(df_w[c]) 
    totals[c]=df_w[c].sum()

for c in ['A','B']:
    df_w[f'cta_{c}']=df_w[f'click_{c}']/df_w[f'N_{c}']

df_w['success']=(df_w['cta_B']>df_w['cta_A'])*1

display(df_w)
print('totals')
display(totals)

print(f"Success rate (= cta in B> cta in A) by day is {round(df_w['success'].sum()/df_w.shape[0]*100,2)}%")

for c in ['A', 'B']:
    totals[f'cta_{c}']=totals[f'click_{c}']/totals[f'N_{c}']

dif=totals['cta_B']-totals['cta_A']
print(f"Difference in metric (exp-control): {round(dif,4)}")


,day,click_A,N_A,click_B,N_B,cta_A,cta_B,success
0,d1,51,1292,115,1305,0.039474,0.088123,1
1,d2,39,853,73,835,0.045721,0.087425,1
2,d3,64,1129,91,1133,0.056687,0.080318,1
3,d4,43,873,60,871,0.049255,0.068886,1
4,d5,55,1197,78,1134,0.045948,0.068783,1
5,d6,44,1023,72,1015,0.043011,0.070936,1
6,d7,56,1003,76,977,0.055833,0.077789,1


totals


{'click_A': 352, 'N_A': 7370, 'click_B': 565, 'N_B': 7270}

Success rate (= cta in B> cta in A) by day is 100.0%
Difference in metric (exp-control): 0.03


In [16]:
d_min=0.01
alpha=0.05
beta=0.2
conf=0.95

emperical_SE=0.0035
emperical_SE_N=10000


    
def sanity_check_single_metric(dif, d_min, emperical_SE, emperical_SE_N, N_A, N_B, conf=0.95, n_tails=2, rnd=4):
    print(f"Observed metric: {round(dif,4)}")
    standard_error=(emperical_SE/math.sqrt(1/emperical_SE_N + 1/emperical_SE_N))*math.sqrt(1/N_B + 1/N_A)
    print(f"Standard error: {round(standard_error,4)}")

    # multiply by z score to get margin of error
    z=st.norm.ppf(1-(1-conf)/n_tails)
    margin_of_error=z*standard_error
    
    # compute confidence interval around 0.5
    interval_min = dif - margin_of_error
    interval_max = dif + margin_of_error

    print("-"*20)
    print(f'Minimal practical significance {round(d_min,4)}')
    print(f"Confidence interval [{round(interval_min,4)},{round(interval_max,4)}]")
    
    print("-"*20)
    #stat significance - interval doesn't contain zero
    if interval_min<0 and interval_max>0:
        print("Confidence interval contains zero. We cannot reject null hypotesis that there is no difference between groups")
    elif interval_min>0:
        print("There is a significant POSITIVE result")
    else:   
        print("There is a significant NEGATIVE result")


    #practical significance   - the interval is to the right from min uplift
    if d_min<=interval_min:
        print(f"With {conf*100}% confidence there is practical and significant effect greater than minimal practical uplift ({d_min}). \nVerdict: LAUNCH the change")
    elif interval_min>=-d_min and interval_max<=d_min:
        print("There is no practical significance. \nVerdict: NO LAUNCH")
    elif interval_min<=-d_min and interval_max>=d_min:
        print ("Overlap. Results do not provide clarity. \nVerdict: Additional test may bring more solid results")
    elif interval_min>=-d_min and interval_max>=d_min:
        print ("Intersection on positive side. Results do not provide clarity. \nVerdict: Additional test may bring more solid results")   
    else: 
        print("NO Launch")
     

In [17]:
sanity_check_single_metric(dif, d_min, emperical_SE, emperical_SE_N, totals['N_A'], totals['N_B'], conf=0.95, n_tails=2, rnd=4)

Observed metric: 0.03
Standard error: 0.0041
--------------------
Minimal practical significance 0.01
Confidence interval [0.0219,0.038]
--------------------
There is a significant POSITIVE result
With 95.0% confidence there is practical and significant effect greater than minimal practical uplift (0.01). 
Verdict: LAUNCH the change
